In [1]:
import numpy as np
import os 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import *
from sklearn.metrics.pairwise import *
import pandas as pd
import warnings
import time
from sklearn import datasets
from scipy.cluster.hierarchy import dendrogram
from numpy import isnan
from pandas import read_csv
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from scipy.stats import *
from sklearn.metrics.pairwise import cosine_distances
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
df_patient_2 = pd.read_excel(r"C:\Users\matti\OneDrive\Desktop\Tirocinio\Genomica\Vivi con ricorrenza\openclinica_rs_ps_pulito.xlsx", header=0)

In [3]:
df_patient = pd.read_excel(r"C:\Users\matti\OneDrive\Desktop\Tirocinio\Genomica\Vivi con ricorrenza\Dataset_rs_ps.xlsx", header=0)

In [7]:
alive_dead = pd.get_dummies(df_patient['follow_Status_of_Patient'], prefix='Status').astype(int)

In [9]:
print(alive_dead)

      Status_Alive  Status_Dead
0                1            0
1                1            0
2                1            0
3                1            0
4                1            0
...            ...          ...
1536             1            0
1537             0            1
1538             0            1
1539             1            0
1540             1            0

[1541 rows x 2 columns]


In [11]:
#INDICE DEI PAZIENTI CHE HANNO AVUTO UNA RECIDIVA DEL TUMORE
index_patient_recurrence = df_patient['follow_Date_of_Recurrence'].dropna().index
len(index_patient_recurrence)

511

In [13]:
# Controlla i valori di "follow_Status_of_Patient" per gli indici di "index_patient_recurrence"
status_values = df_patient.loc[index_patient_recurrence, 'follow_Status_of_Patient']
print(status_values)

5       Alive
8        Dead
14      Alive
20       Dead
21       Dead
        ...  
1528     Dead
1529     Dead
1531     Dead
1535    Alive
1537     Dead
Name: follow_Status_of_Patient, Length: 511, dtype: object


In [15]:
status_counts = status_values.value_counts()
print(status_counts)

follow_Status_of_Patient
Dead     355
Alive    156
Name: count, dtype: int64


In [17]:
# Applica la codifica one-hot alla Serie
recurrence = pd.get_dummies(status_values, prefix='Recurrence').astype(int)
print(recurrence)


      Recurrence_Alive  Recurrence_Dead
5                    1                0
8                    0                1
14                   1                0
20                   0                1
21                   0                1
...                ...              ...
1528                 0                1
1529                 0                1
1531                 0                1
1535                 1                0
1537                 0                1

[511 rows x 2 columns]


In [19]:
# Crea una nuova colonna 'Recurrence' con valore predefinito 0
df_patient['Recurrence_Tumor'] = 0
# Assegna il valore 1 agli indici in index_patient_recurrence
df_patient.loc[index_patient_recurrence, 'Recurrence_Tumor'] = 1


In [21]:
# Crea un DataFrame vuoto con tutti gli indici di df_patient
complete_recurrence = pd.DataFrame(0, index=df_patient.index, columns=['Recurrence_Alive', 'Recurrence_Dead'])

# Assegna i valori presenti in recurrence ai rispettivi indici
complete_recurrence.loc[recurrence.index, ['Recurrence_Alive', 'Recurrence_Dead']] = recurrence

df_patient = pd.concat([df_patient, complete_recurrence], axis=1)

In [23]:
df_patient = pd.concat([df_patient, alive_dead], axis=1)
df_patient.columns

Index(['Unnamed: 0', 'Patient_ID', 'Study_Name', 'Study_OID',
       'Study_Start_Date', 'chemo_Best_Tumor_Response',
       'chemo_Cancer_Therapy_Agent_Name', 'chemo_Chemotherapy_Treatment',
       'chemo_Note_chemo', 'chemo_Number_of_Cycles_Performed_1',
       ...
       'qol_EQ-5D-5L_24M_Self-Care', 'qol_EQ-5D-5L_24M_Usual_Activities',
       'qol_EQ-5D-5L_24M_Pain/Discomfort',
       'qol_EQ-5D-5L_24M_Anxiety/Depression', 'qol_EQ-5D-5L_24M_Health_Today',
       'Recurrence_Tumor', 'Recurrence_Alive', 'Recurrence_Dead',
       'Status_Alive', 'Status_Dead'],
      dtype='object', length=563)

In [25]:
patient = df_patient.loc[index_patient_recurrence, 'Patient_ID']
patient = np.array(patient)
patient

array(['SS_UL0006', 'SS_UL0009', 'SS_UL0015', 'SS_UDUS115', 'SS_UDUS116',
       'SS_UDUS125', 'SS_UDUS126', 'SS_UDUS129', 'SS_UDUS133',
       'SS_UDUS148', 'SS_UDUS163', 'SS_UDUS140', 'SS_MAASTRO_5588',
       'SS_MAASTRO_215', 'SS_MAASTRO_416', 'SS_MAASTRO_588',
       'SS_MAASTRO_4746', 'SS_MAASTRO_2867', 'SS_MAASTRO_7077',
       'SS_MAASTRO_177', 'SS_MAASTRO_7265', 'SS_MAASTRO_5082',
       'SS_MAASTRO_8441', 'SS_MAASTRO_8118', 'SS_MAASTRO_8386',
       'SS_MAASTRO_6905', 'SS_INT_MI_0045', 'SS_INT_MI_0146',
       'SS_INT_MI_0103', 'SS_INT_MI_0055', 'SS_INT_MI_0034',
       'SS_INT_0266', 'SS_INT_0056', 'SS_INT_0064', 'SS_INT_0105',
       'SS_INT_0148', 'SS_INT_0157', 'SS_INT_0016', 'SS_INT_0017',
       'SS_INT_0020', 'SS_INT_0069', 'SS_INT_0267', 'SS_INT_0138',
       'SS_INT_0197', 'SS_INT_0272', 'SS_INT_0220', 'SS_INT_0236',
       'SS_INT_0247', 'SS_INT_0144', 'SS_INT_0182', 'SS_INT_0024',
       'SS_INT_0249', 'SS_INT_0003', 'SS_INT_0313', 'SS_INT_0012',
       'SS_INT_030

In [27]:
df_patient_filtered = df_patient.loc[:, df_patient.isnull().sum() == 0]
df_patient_filtered.columns

Index(['Unnamed: 0', 'Patient_ID', 'Study_Name', 'Study_OID',
       'Study_Start_Date', 'clinical_Sex', 'Recurrence_Tumor',
       'Recurrence_Alive', 'Recurrence_Dead', 'Status_Alive', 'Status_Dead'],
      dtype='object')

In [29]:
df_patient_filtered_2 = df_patient_2.loc[:, df_patient_2.isnull().sum() == 0]
df_patient_filtered_2.columns

Index(['Unnamed: 0', 'Study_Start_Date', 'clinical_Age_at_Diagnosis',
       'clinical_Date_of_first_Diagnosis', 'clinical_Sex',
       'ctn_Anatomical_Tumor_Location', 'ctn_Laterality_of_T',
       'ctn_Stage_at_Diagnosis_7Edition', 'ctn_TNM_cN_7Edition',
       'ctn_TNM_cT_7Edition', 'ctn_Tumor_Region',
       'follow_Date_of_Examination_or_Last_Contact', 'follow_Follow_Up_Period',
       'follow_Recurrence', 'follow_Status_of_Patient'],
      dtype='object')

In [88]:
df_patient.to_excel(os.path.join(r"C:\Users\matti\OneDrive\Desktop\Tirocinio\Genomica\Vivi con ricorrenza", 'Dataset_rs_ps_ricorrenza.xlsx'), index=False)

In [31]:
filtered_df = df_patient[df_patient['Patient_ID'].isin(patient)]

print(filtered_df.index)

Index([   5,    8,   14,   20,   21,   28,   29,   31,   35,   45,
       ...
       1505, 1507, 1510, 1511, 1526, 1528, 1529, 1531, 1535, 1537],
      dtype='int64', length=511)


In [33]:
#VOGLIO STAMPARE LE COLONNE DEI PAZIENTI CHE HANNO AVUTO UNA RECIDIVA DEL TUMORE CHE NON HANNO MISSING VALUES
col_filt = filtered_df.loc[:, filtered_df.isnull().sum() == 0]
col_filt.columns

Index(['Unnamed: 0', 'Patient_ID', 'Study_Name', 'Study_OID',
       'Study_Start_Date', 'clinical_Age_at_Diagnosis',
       'clinical_Date_of_first_Diagnosis', 'clinical_Hospital', 'clinical_Sex',
       'ctn_Anatomical_Tumor_Location', 'ctn_Laterality_of_T',
       'ctn_Stage_at_Diagnosis_7Edition', 'ctn_TNM_cN_7Edition',
       'ctn_TNM_cT_7Edition', 'ctn_Tumor_Region',
       'follow_Date_of_Examination_or_Last_Contact',
       'follow_Date_of_Recurrence', 'follow_Follow_Up_Period',
       'follow_Recurrence', 'follow_Status_of_Patient',
       'follow_Treatment_of_Recurrence', 'follow_Type_of_Recurrence',
       'patho_HPV_Status_Evaluated', 'patient_Consent',
       'patient_Eligibility_Criteria', 'risk_Additional_precancerous_lesion_1',
       'risk_Alcohol_at_Time_of_Diagnosis',
       'risk_Family_History_of_Malignancies', 'risk_Oral_Hygiene',
       'risk_Smoker_at_Time_of_Diagnosis', 'Recurrence_Tumor',
       'Recurrence_Alive', 'Recurrence_Dead', 'Status_Alive', 'Status_De

In [39]:
col_filt.to_excel(os.path.join(r"C:\Users\matti\OneDrive\Desktop\Tirocinio\Genomica\Vivi con ricorrenza", 'Dataset_pazienti_con_recidiva.xlsx'), index=False)